In [1]:
import os
import cv2
import numpy as np
import pandas as pd

In [2]:
def bezier_curve(t, P0, P1, P2, P3, P4):
    return (
        (1 - t) ** 4 * P0 +
        4 * (1 - t) ** 3 * t * P1 +
        6 * (1 - t) ** 2 * t ** 2 * P2 +
        4 * (1 - t) * t ** 3 * P3 +
        t ** 4 * P4
    )

# Modified function to check Bézier validity
def check_invalid_bezier(row):
    try:
        x_min, y_min, x_max, y_max = eval(row["BoundingBox in Crop"])
        w, h = x_max - x_min, y_max - y_min
    
        P0 = np.array(eval(row["P0"]))
        P1 = np.array(eval(row["P1"]))
        P2 = np.array(eval(row["P2"]))
        P3 = np.array(eval(row["P3"]))
        P4 = np.array(eval(row["P4"]))
    
        x_P0, y_P0 = P0
        x_P1, y_P1 = P1
        x_P2, y_P2 = P2
        x_P3, y_P3 = P3
        x_P4, y_P4 = P4
    
        if w < 250:
            if (x_P4 - x_P0) < 0.7 * w:
                raise ValueError(f"{row['Crop File Name']}: Horizontal span incorrect 1")
        elif w < 750:
            if (x_P4 - x_P0) < 0.9 * w:
                raise ValueError(f"{row['Crop File Name']}: Horizontal span incorrect 2")
        else:
            if (x_P4 - x_P0) < 0.95 * w:
                raise ValueError(f"{row['Crop File Name']}: Horizontal span incorrect 3")

        if h > 120:
            if max(y_P0, y_P1, y_P2, y_P3, y_P4) - min(y_P0, y_P1, y_P2, y_P3, y_P4) < 0.7 * h:
                raise ValueError(f"{row['Crop File Name']}: Vertical height incorrect")
    
        if (y_P1 > y_P0) and (w / h > 2):
            raise ValueError(f"{row['Crop File Name']}: P1 lower than start point")
        if (y_P2 < y_min - h) or (y_P2 > y_max + h):
            raise ValueError(f"{row['Crop File Name']}: P2 too far from bbox")
        if (y_P3 > y_P4) and (w / h > 2):
            raise ValueError(f"{row['Crop File Name']}: P3 lower than end point")
        if (x_P1 < x_P0) or (x_P1 > x_P2):
            raise ValueError(f"{row['Crop File Name']}: P1 not between P0 and P2")
        if (x_P2 < x_P1) or (x_P2 > x_P3):
            raise ValueError(f"{row['Crop File Name']}: P2 not between P1 and P3")
        if (x_P3 < x_P2) or (x_P3 > x_P4):
            raise ValueError(f"{row['Crop File Name']}: P3 not between P2 and P4")
    
        image_path = os.path.join("Use", row["Crop File Name"])
        if not os.path.exists(image_path):
            raise FileNotFoundError(f"{row['Crop File Name']}: Image not found")
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if image is None:
            raise ValueError(f"{row['Crop File Name']}: Could not read image")
    
        # Sample points along the Bézier curve
        t_vals = np.linspace(0, 1, 200)
        bezier_points = np.array([bezier_curve(t, P0, P1, P2, P3, P4) for t in t_vals])
        bezier_points = bezier_points.astype(int)  # Convert to integer indices
    
        black_pixel_count = 0  # Count black pixels along the Bézier path
        for x, y in bezier_points:
            if 0 <= y < image.shape[0] and 0 <= x < image.shape[1]:
                if image[y, x] < 128:  # PARAMETER
                    black_pixel_count += 1
        if w < 300:
            if black_pixel_count / len(bezier_points) < 0.95:
                raise ValueError(f"{row['Crop File Name']}: Part of Bezier path is empty 1")
        elif w < 800:
            if black_pixel_count / len(bezier_points) < 0.8:
                raise ValueError(f"{row['Crop File Name']}: Part of Bezier path is empty 2")
        elif w < 1600:
            if black_pixel_count / len(bezier_points) < 0.7:
                raise ValueError(f"{row['Crop File Name']}: Part of Bezier path is empty 3")
        else:
            if black_pixel_count / len(bezier_points) < 0.6:
                raise ValueError(f"{row['Crop File Name']}: Part of Bezier path is empty 4")
    
        return 0

    except Exception as e:
        with open(error_log_path, "a") as error_log:
            error_log.write(str(e) + "\n")
        return 1

In [3]:
control_points_path = "control_points.csv"
df = pd.read_csv(control_points_path)
error_log_path = "curve_error.log"

df["Invalid Bezier Curve"] = 0
df["Invalid Bezier Curve"] = df.apply(check_invalid_bezier, axis=1)

df.to_csv(control_points_path, index=False)

print(f"Updated control_points.csv with invalid Bézier curve checks.")

Updated control_points.csv with invalid Bézier curve checks.
